In [1]:
# Loading required libraries
import csv
import sys
import pandas as pd
import os
import glob
import itertools
import numpy as np
import copy
import math
import time
from pathlib import Path
from collections import Counter

# Loading trace : Needs to expanded into 4K chunks
path = r'/Users/chandranilchakraborttii/Documents/GC_pred/data'
all_files = glob.glob(os.path.join(path, "ssdtrace-01_dataprep_complete.csv_deathtime_added_Heiner.csv"))

f = all_files[0]  # Change the file name as required
print("File " + str(f))
cols = ['io','LBA','deathtime']
df = pd.read_csv(f,engine='python',skiprows =1,header=None,na_values=['-1'], index_col=False)
df.columns = cols
df['deathtime'] = df['deathtime'].replace(np.NaN, -1)
lba_list = df['LBA'].tolist()
print("Min LBA in the dataset :", min(lba_list))
print("Max LBA in the dataset :", max(lba_list))
print("Number of unique LBAs in the data :",len(Counter(df['LBA'])))
print("Number of IO Accesses :",len(df))

File /Users/chandranilchakraborttii/Documents/GC_pred/data/ssdtrace-01_dataprep_complete.csv_deathtime_added_Heiner.csv
Min LBA in the dataset : 0
Max LBA in the dataset : 998900152
Number of unique LBAs in the data : 179168
Number of IO Accesses : 1062003


In [2]:
#SSD specifications
num_page_addresses = len(Counter(df['LBA']))
page_size = 4096
page_per_block = 64    
GB = 1024*1024*1024
SSD_size = num_page_addresses*page_size
# SSD_size_GB_normal = round_decimals_up(SSD_size/GB,1)
SSD_size_GB_normal = SSD_size/GB
over_provisioning_ratio = 0.2
LOG_PAGE_PER_BLOCK = int(math.log(page_per_block,2))
# SSD_size_full = round_decimals_up((1 + over_provisioning_ratio)*SSD_size_GB_normal,1)
SSD_size_full = (1 + over_provisioning_ratio)*SSD_size_GB_normal
print("SSD Capacity (Available in GB) :",SSD_size_GB_normal)
print("SSD Capacity (Total in GB)     :",SSD_size_full)

SSD Capacity (Available in GB) : 0.6834716796875
SSD Capacity (Total in GB)     : 0.820166015625


In [3]:
GB = 1024*1024*1024
ssd_capacity = SSD_size_full *GB
page_addresses = []
block_addresses = []
block_placement = 0
start_counter = -1
block_addresses.append(0)

while(start_counter < (ssd_capacity/page_size) - page_size):
    start_counter = start_counter + 1
    page_addresses.append(int(start_counter))
    if(block_placement >= page_per_block):
        block_addresses.append(int(start_counter))
        block_placement = 0

    block_placement = block_placement + 1

free_list_block = copy.deepcopy(block_addresses)
free_list_page = copy.deepcopy(page_addresses)

# Defining block_structure
valid_bitmap = []
write_ptr=0
invalid_pages=0
block_struct = {}

for x in free_list_block:
    start_lba = x
    valid_bitmap = []
    for x in range(page_per_block):
        valid_bitmap.append(False)
    segment = [start_lba,invalid_pages,valid_bitmap,write_ptr]
    block_struct[start_lba]=segment
    

if(len(page_addresses)*0.75 < len(Counter(df['LBA']))):
    print("WARNING...! Not enough blocks. Need to increase SSD Size")
else:
    print("Looks good.Go ahead!")

WARNING...! Not enough blocks. Need to increase SSD Size


In [4]:
def invalidate_lba(lba):
    prev = L2P[lba]
    prev_block = (prev >> LOG_PAGE_PER_BLOCK)*page_per_block
    prev_page = prev % page_per_block
    block_details = block_struct[prev_block]                                 # Getting block details
    block_struct[prev_block][2][prev_page] = False                           # Setting bitmap to False
    block_struct[prev_block][1] = block_struct[cur_block][2].count(True)     # Setting invalid pages
    L2P.pop(lba)
    

def map_lba(lba,cur_block):
    phys_addr = block_struct[cur_block][0] + block_struct[cur_block][3]
    L2P[lba] = phys_addr
    P2L[phys_addr] = lba   
    block_struct[cur_block][2][block_struct[cur_block][3]] =  True               # Setting Bitmap
    block_struct[cur_block][1] =  block_struct[cur_block][2].count(True)         # Setting invalid pages
    block_struct[cur_block][3] = block_struct[cur_block][3] + 1                  # Increasing Write pointer
    total_writes[0] = total_writes[0] + 1
    
def check_GC (cur_block, in_gc):
    if (block_struct[cur_block][3] < page_per_block):
        return cur_block
    else:
        closed_blocks.append(cur_block)
        if(len(free_list_block) <= 0):
            print("FAIL WHILE DOING GC, RAN OUT OF BLOCKS") 
        elif (len(free_list_block) <= GC_THRESHOLD):
            if(in_gc != True):
                if(block_struct[cur_block][3] >= page_per_block):
                    cur_block = free_list_block.pop(0)
                in_gc = do_greedy_gc(cur_block,in_gc)
        if(block_struct[cur_block][3] == (page_per_block)):
            cur_block = free_list_block.pop(0)
        return cur_block


def do_greedy_gc(cur_block,in_gc):
    in_gc = True
    gc_writes = 0
    min_val = float('inf')  
    for x in closed_blocks:               
        if (block_struct[x][1] < min_val):
            min_val = block_struct[x][1]
            gc_blk = x

    
    #found the block with minimal valid pages, move all valid pages
    for pg in range(page_per_block):
        #figure out the logical addresses for all phys pages in the gc block
        phys_addr = block_struct[gc_blk][0] + pg
        gc_lba = P2L[phys_addr]
        
        # Checking for valid bitmap
        prev_block = (phys_addr >> LOG_PAGE_PER_BLOCK)*page_per_block
        prev_page = phys_addr % page_per_block
        bitmap = block_struct[prev_block][2][prev_page]
        # If valid bitmap is True (data is valid), copy to OP capacity, increase GC writes
        if (bitmap == True):
            invalidate_lba(gc_lba)
            #check if we need to get a new block
            cur_block = check_GC(cur_block,in_gc)
            #move the gc'ed block t-o a new location
            map_lba(gc_lba,cur_block)   
            gc_writes = gc_writes + 1
           
            
    if(gc_writes > 64):
        print("GC writes not as expected", gc_writes)
        
    # Reset block details, remove from closed list and add to free_list
    invalid_pages = 0
    valid_bitmap = []
    write_ptr = 0
    for x in range(page_per_block):
        valid_bitmap.append(False)
        
    block_struct[gc_blk]= [gc_blk,invalid_pages,valid_bitmap,write_ptr]
    closed_blocks.remove(gc_blk)
    free_list_block.append(gc_blk)
    total_gc_writes[0] = total_gc_writes[0] + gc_writes
    in_gc = False
    return in_gc

In [5]:
L2P = {}
P2L = {}
total_gc_writes = []
total_gc_writes.append(0)
counter = 0
GC_THRESHOLD = 100
min_LBA = min(lba_list)
closed_blocks = []
cur_block = free_list_block.pop(0)
block_details = block_struct[cur_block]
global in_gc 
in_gc = False
global gc_writes
gc_writes = 0
total_writes = []
total_writes.append(0)


start_time = time.time()
while(counter < len(lba_list)):
    lba=int(lba_list[counter]) - min_LBA
    if(counter >100000 and counter%100000==0):
        print("Percentage completed in (%)  :", (counter/len(lba_list))*100)
    if lba in L2P:
        invalidate_lba(lba)
    cur_block = check_GC(cur_block,in_gc)
    map_lba(lba,cur_block)
    counter = counter + 1
    
end_time = time.time()
run_time = end_time - start_time
print("Trace Replay Ended!")

Percentage completed in (%)  : 18.832338515051276
Percentage completed in (%)  : 28.248507772576914
Percentage completed in (%)  : 37.66467703010255
Percentage completed in (%)  : 47.08084628762819
Percentage completed in (%)  : 56.49701554515383
Percentage completed in (%)  : 65.91318480267947
Percentage completed in (%)  : 75.3293540602051
Percentage completed in (%)  : 84.74552331773074
Percentage completed in (%)  : 94.16169257525638
Trace Replay Ended!


In [8]:
print("GC Threshold set",GC_THRESHOLD)
print("OverProvisioning Ratio :",over_provisioning_ratio)
print("Total Number of Writes :",(total_writes[0]))
print("Total Number of user writes :",(total_writes[0] - total_gc_writes[0]))
print("Total Number of GC writes :",total_gc_writes[0])
print("Write Amplification:  ",total_writes[0]/(total_writes[0] - total_gc_writes[0]))
print("Execution Time for the FTL :",run_time)
print("Done")

GC Threshold set 100
OverProvisioning Ratio : 0.2
Total Number of Writes : 3214074
Total Number of user writes : 1062003
Total Number of GC writes : 2152071
Write Amplification:   3.0264264790212456
Execution Time for the FTL : 21.498674869537354
Done
